In [0]:
from keras.models import Sequential
from keras.layers import GRU,Dense,Dropout
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import keras


In [176]:
data=pd.read_csv("market_price_analysis.csv")
print(data.head())
data.shape

         Date    Open    High     Low   Close   Volume  OpenInt
0  1962-01-02  0.6277  0.6362  0.6201  0.6201  2575579        0
1  1962-01-03  0.6201  0.6201  0.6122  0.6201  1764749        0
2  1962-01-04  0.6201  0.6201  0.6037  0.6122  2194010        0
3  1962-01-05  0.6122  0.6122  0.5798  0.5957  3255244        0
4  1962-01-08  0.5957  0.5957  0.5716  0.5957  3696430        0


(14058, 7)

In [177]:
print(data['OpenInt'].unique())
print(data.describe())
data=data.drop(['OpenInt','Date'],axis=1)

[0]
               Open          High  ...        Volume  OpenInt
count  14058.000000  14058.000000  ...  1.405800e+04  14058.0
mean      10.534737     10.642538  ...  2.423066e+07      0.0
std       11.742897     11.868891  ...  3.369859e+07      0.0
min        0.459020      0.466900  ...  0.000000e+00      0.0
25%        0.901500      0.909600  ...  5.496950e+06      0.0
50%        3.880900      3.912700  ...  1.597520e+07      0.0
75%       21.416000     21.658000  ...  2.853667e+07      0.0
max       47.751000     48.459000  ...  9.345354e+08      0.0

[8 rows x 6 columns]


In [178]:
####
print(data.columns)


Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


In [179]:
dataY=data.pop('Volume')
print(dataY.head())
print(data.head())

0    2575579
1    1764749
2    2194010
3    3255244
4    3696430
Name: Volume, dtype: int64
     Open    High     Low   Close
0  0.6277  0.6362  0.6201  0.6201
1  0.6201  0.6201  0.6122  0.6201
2  0.6201  0.6201  0.6037  0.6122
3  0.6122  0.6122  0.5798  0.5957
4  0.5957  0.5957  0.5716  0.5957


In [180]:
dataX=data
print(dataX.head())
print(dataY.head())

     Open    High     Low   Close
0  0.6277  0.6362  0.6201  0.6201
1  0.6201  0.6201  0.6122  0.6201
2  0.6201  0.6201  0.6037  0.6122
3  0.6122  0.6122  0.5798  0.5957
4  0.5957  0.5957  0.5716  0.5957
0    2575579
1    1764749
2    2194010
3    3255244
4    3696430
Name: Volume, dtype: int64


In [0]:
#####Scale the columns
datay=dataY.values.astype(float).reshape(-1,1)
dataY_scaled=MinMaxScaler().fit_transform(datay)


In [182]:
def test_train_split(test_size=0.2):
  index=int(len(data) * test_size)
  train_x=dataX[0:index]
  train_y=dataY_scaled[0:index]
  test_x=dataX[index:]
  test_y=dataY_scaled[index:]
  return train_x,train_y,test_x,test_y

  
train_x,train_y,test_x,test_y=test_train_split(test_size=0.2)

print(train_x.head())

     Open    High     Low   Close
0  0.6277  0.6362  0.6201  0.6201
1  0.6201  0.6201  0.6122  0.6201
2  0.6201  0.6201  0.6037  0.6122
3  0.6122  0.6122  0.5798  0.5957
4  0.5957  0.5957  0.5716  0.5957


In [0]:
trains_x=train_x.values.reshape((-1,1,4))


In [0]:
def build_model():
  model = Sequential()
  model.add(GRU(units=50,activation='tanh',return_sequences=True,input_shape=(1,4)))
  #model.add(Dropout(0.2))
  model.add(GRU(units=50))
  #model.add(Dropout(0.2))
  model.add(Dense(1))
  model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
  return model

In [0]:
from keras.callbacks import EarlyStopping
abc=EarlyStopping(monitor='val_loss',patience=3)



In [206]:
model=build_model()
model.fit(trains_x,train_y,validation_split=0.1,epochs=100,batch_size=32,callbacks=[abc])
model.save('gru.h5')

Train on 2529 samples, validate on 282 samples
Epoch 1/100
2529/2529 [==============================] - 6s 2ms/step - loss: 5.3485e-05 - acc: 0.0000e+00 - val_loss: 2.4679e-06 - val_acc: 0.0000e+00
Epoch 2/100
2529/2529 [==============================] - 1s 366us/step - loss: 4.5233e-06 - acc: 0.0000e+00 - val_loss: 3.8700e-06 - val_acc: 0.0000e+00
Epoch 3/100
2529/2529 [==============================] - 1s 379us/step - loss: 4.7408e-06 - acc: 0.0000e+00 - val_loss: 1.2813e-05 - val_acc: 0.0000e+00
Epoch 4/100
2529/2529 [==============================] - 1s 378us/step - loss: 5.0051e-06 - acc: 0.0000e+00 - val_loss: 3.3377e-06 - val_acc: 0.0000e+00


In [0]:
from keras.models import load_model
model=load_model('gru.h5')
tests_x=test_x.values.reshape((-1,1,4))

